In [1]:
%matplotlib widget

In [2]:
from functools import partial

In [3]:
from libschrodinger.crank_nicolson_2d import *

In [4]:
from libschrodinger.potentials import *

In [5]:
import numpy as np

In [6]:
import pandas as pd

In [7]:
from pathlib import Path

In [8]:
from typing import Dict

In [9]:
caseName : str = "lightspeed1"

In [10]:
math = np
spatialStep : float = .01
length : float = 4
temporalStep : float = (spatialStep ** 2) / 4
courantNumber : float = 1.0
pointCount : int = int(length / spatialStep)
potentialHeight : float = 200
preRegionLength = .3
preRegionScalar = 2
packetStandardDeviation : float = preRegionLength / preRegionScalar
packetStartX : float = 1 - ((preRegionScalar / 2) * packetStandardDeviation)
packetStartY : float = .5
packetWaveNumber : float = 13 * np.pi
barrierCount : int = 3
postRegionLength = .3
intraRegionLength : float = 1 - (postRegionLength + preRegionLength)
barrierWidth : float = intraRegionLength / barrierCount
constantPotentialLengthRatios : List[float] = [postRegionLength] \
        + ([barrierWidth] * barrierCount) \
        + [preRegionLength]
print("Point Count: ", pointCount)
print("Post Region Length: ", postRegionLength)
print("Intra Region Length: ", intraRegionLength)
print("Pre Region Length: ", preRegionLength)
print("Barrier Length: ", barrierWidth)
print("Total Length: ", intraRegionLength + postRegionLength + preRegionLength)
print("Total Length Ratios: ", np.sum(constantPotentialLengthRatios))
frameCount : int = 650
backend = "numpy"

Point Count:  400
Post Region Length:  0.3
Intra Region Length:  0.4
Pre Region Length:  0.3
Barrier Length:  0.13333333333333333
Total Length:  1.0
Total Length Ratios:  1.0


In [11]:
wavePacketFunction = lambda position : makeWavePacket(
            position, 
            packetStartX * length, 
            packetStartY * length, 
            spatialStep, 
            packetStandardDeviation, 
            packetWaveNumber, 
            math
        )

In [12]:
def recordTotalLengthWiseProbabilities(
            simulator : Simulator, 
            regionLengths : List[float], 
            regionLabels : Tuple[str], 
            math = np, 
            regionLabelPrepend = "TotalProbability::"
        ) -> pd.DataFrame:
    assert len(regionLabels) == len(regionLengths)
    currentPosition : float = 0
    regionalProbabilities : List[np.array] = []
    regionalCutFrames : List[np.array] = []
    for ii in range(len(regionLengths)): 
        regionLength = regionLengths[ii]
        probabilities, cutFrames = totalProbabilityInRegion(
                simulator.probabilities, 
                simulator.grid.pointCount, 
                simulator.spaceStep, 
                currentPosition, 
                0, 
                regionLength, 
                1, 
                math
            )
        regionalProbabilities.append(probabilities)
        regionalCutFrames.append(cutFrames)
        currentPosition += regionLength
    totalProbabilities = {
            (regionLabelPrepend + "Probabilities::" + regionLabels[ii]) : regionalProbabilities[ii] \
            for ii in range(len(regionLengths))
        }
    return totalProbabilities, regionalCutFrames

In [13]:
def logConstantMeasurementRegionSimulation(
            baseName : str, 
            length : float, 
            simulator : Simulator, 
            simulationCount : int, 
            allData : Dict, 
            constantRegionLengths : List[float], 
            constantRegionLabels : List[str], 
            showWhenSimulationDone : bool = False, 
            math = np, 
            basePath = None, 
            animationInterval = 30
        ):
    basePath = basePath if basePath else Path.cwd() / baseName
    if showWhenSimulationDone == True: 
        print("Simulation " + str(simulationCount) + ": done processing probabilities.")
    if showWhenSimulationDone == True: 
        print("Simulation " + str(simulationCount) + ": logging.")
    name = baseName + str(simulationCount) + "::"
    if showWhenSimulationDone == True: 
        print("Saving Video of " + name[:-2])
    videoPath = basePath / str(simulationCount)
    videoPath.mkdir(parents = True, exist_ok = True)
    totalProbabilities, cutFrames = recordTotalLengthWiseProbabilities(
            simulator, 
            constantRegionLengths, 
            constantRegionLabels, 
            math, 
            name
        )
    waveAnimation = animateImages(
            length, 
            simulator.probabilities, 
            animationInterval, 
            0, 
            math.max(simulator.probabilities), 
            constantRegionLengths, 
            [1] * len(constantRegionLengths), 
            colorMap = "hot"
        )
    waveAnimation.save(str(videoPath / (str(simulationCount) + ".gif")))
    plt.close()
    plt.figure()
    plt.imshow(simulator.potentials[0])
    plt.savefig(str(videoPath / (str(simulationCount) + "Potential.png")))
    plt.close()
    totalProbabilities[name + "FrameCount"] = frameCount
    totalProbabilities[name + "SpaceStep"] = simulator.profile.spaceStep
    totalProbabilities[name + "TimeStep"] = simulator.profile.timeStep
    totalProbabilities[name + "PointCount"] = simulator.profile.grid.pointCount
    for ii in range(len(constantRegionLabels)): 
        print("Saving Video of " + constantRegionLabels[ii])
        totalProbabilities[name + "RegionLength::" + constantRegionLabels[ii]] = constantRegionLengths[ii]
        cutAnimation = animateImages(
                length * constantRegionLengths[ii], 
                cutFrames[ii], 
                animationInterval, 
                0, 
                math.max(cutFrames[ii]), 
                colorMap = "hot"
            )
        cutAnimation.save(str(videoPath / (constantRegionLabels[ii] + ".gif")))
        plt.close()
    allData |= totalProbabilities
    if showWhenSimulationDone == True: 
        print("Done logging " + name[:-2])
    simulationCount += 1
    if showWhenSimulationDone == True: 
        print("Producing Simulation final output CSV")
    return allData
    

In [14]:
def constantSimulationProfiles( 
            initialWaveFunction, 
            spatialStep : float, 
            temporalStep : float,
            length : float, 
            regionLengthRatios : List[float], 
            regionPotentialRatios : List[List[float]], 
            simulateControl : bool, 
            math = np, 
            gpuAccelerated = False, 
            edgeBound = False, 
            useDense = False, 
            courantNumber = 1.0
        ) -> List[SimulationProfile]:
    if simulateControl == True: 
        regionPotentialRatios.append([0.0 for ii in range(len(regionPotentialRatios[0]))])
    profiles : List[SimulationProfile] = []
    potentialFunction = lambda potentialRatios, position, time : constantPotentials(
                position, 
                math.array(regionLengthRatios), 
                potentialRatios, 
                potentialHeight, 
                math
        )
    for potentialRatios in regionPotentialRatios: 
        profile = SimulationProfile(
            makeLinspaceGrid(pointCount, length, 2, False, float, math), 
            initialWaveFunction, 
            partial(potentialFunction, potentialRatios), 
            temporalStep, 
            spatialStep, 
            constantPotential = True, 
            gpuAccelerated = gpuAccelerated, 
            edgeBound = edgeBound, 
            useDense = useDense, 
            courantNumber = courantNumber, 
            length = length
        )
        profiles.append(profile)
    return profiles

In [15]:
def recordConstantRegionSimulations(
            profiles : List[SimulationProfile], 
            frames : int, 
            baseName : str, 
            measurmentRegions : List[float], 
            showWhenSimulationDone = False, 
            discardSimulations = True, 
            constantRegionLabels : List[str] = None, 
            basePath = None, 
            animationInterval = 30, 
            showBar : bool = False, 
            showFPS : bool = False, 
            showTotalTime : bool = False, 
            math = np
        ):
    simulations : List[Simulator] = []
    simulationCount : int = 0
    allData = {}
    constantRegionLabels \
            = ["Region" + str(ii) for ii in range(len(constantRegionLabels))] \
            if constantRegionLabels == None else constantRegionLabels 
    for profile in profiles: 
        simulator = Simulator(profile)
        simulator.simulate(frames, showBar, showFPS, showTotalTime)
        if showWhenSimulationDone == True: 
            print("Simulation " + str(simulationCount) + " is done, processing probabilities.")
        probabilities, probabilityDecibles = simulator.processProbabilities()
        if discardSimulations == False: 
            simulations.append(simulator)
        allData = logConstantMeasurementRegionSimulation(
            baseName, 
            simulator.profile.length, 
            simulator, 
            simulationCount, 
            allData, 
            measurmentRegions, 
            constantRegionLabels, 
            showWhenSimulationDone, 
            math, 
            basePath, 
            animationInterval
        )
        simulationCount += 1
    return allData, simulations

In [16]:
potentials : List[List[float]] = [
        [0, 0, 0, 0, 0]
    ]

In [17]:
profiles : List[SimulationProfile] = constantSimulationProfiles(
        wavePacketFunction, 
        spatialStep, 
        temporalStep, 
        length, 
        constantPotentialLengthRatios, 
        potentials, 
        False, 
        edgeBound = True
    )

In [18]:
constantPotentialLengthRatios

[0.3, 0.13333333333333333, 0.13333333333333333, 0.13333333333333333, 0.3]

In [19]:
allData, _ = recordConstantRegionSimulations(
        profiles, 
        frameCount, 
        caseName, 
        constantPotentialLengthRatios, 
        True, 
        constantRegionLabels = ["Post", "3rdStair", "2ndStair", "1stStair", "Pre"], 
        showBar = True, 
        showFPS = True, 
        showTotalTime = True
    )

[====================================================================================================]
[----------------------------------------------------------------------------------------------------]
Total Time:  1238.1434135620002
Frames Per Second:  0.5401958176701046
Simulation 0 is done, processing probabilities.
Simulation 0: done processing probabilities.
Simulation 0: logging.
Saving Video of lightspeed00
Saving Video of Post
Saving Video of 3rdStair
Saving Video of 2ndStair
Saving Video of 1stStair
Saving Video of Pre
Done logging lightspeed00
Producing Simulation final output CSV


In [20]:
allData["packetStartX"] = packetStartX
allData["packetStartY"] = packetStartY
allData["initialWaveNumber"] = packetWaveNumber
allData["standardDeviation"] = packetStandardDeviation

In [21]:
pd.DataFrame(allData).to_csv(str(Path.cwd() / caseName / (caseName + ".csv")))